In [1]:
import os
import duckdb

In [2]:
def list_file_size(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            size_mb = os.path.getsize(fp) / (1024 * 1024)
            print(f"{fp}: {size_mb:.2f} MB")

def export_mmsi_to_parquet(con, table, mmsi, out_dir="output"):
    os.makedirs(out_dir, exist_ok=True)
    out_file = os.path.join(out_dir, f"mmsi_{mmsi}.parquet")

    con.execute(f"""
        COPY (
            SELECT *
            FROM {table}
            WHERE MMSI = {mmsi}
        )
        TO '{out_file}'
        (FORMAT PARQUET, COMPRESSION ZSTD);
    """)

    print(f"Exported MMSI {mmsi} → {out_file}")

In [3]:
con = duckdb.connect("ais.duckdb")

In [4]:
feb_distinct_mmsi = con.execute("""
    SELECT DISTINCT MMSI
    FROM feb_ais
    WHERE MMSI IS NOT NULL AND MMSI <> 0
""").fetchdf()

print("Unique Feb MMSI count:", len(feb_distinct_mmsi))

Unique Feb MMSI count: 14560


In [5]:
aug_distinct_mmsi = con.execute("""
    SELECT DISTINCT MMSI
    FROM aug_ais
    WHERE MMSI IS NOT NULL AND MMSI <> 0
""").fetchdf()

print("Unique Aug MMSI count:", len(aug_distinct_mmsi))

Unique Aug MMSI count: 18790


In [6]:
feb_top20_mmsi = con.execute("""
    SELECT MMSI,
           COUNT(*) AS n_row,
           MIN("TIMESTAMP") AS first_ts,
           MAX("TIMESTAMP") AS last_ts
    FROM feb_ais
    WHERE MMSI IS NOT NULL AND MMSI <> 0
    GROUP BY MMSI
    ORDER BY n_row DESC
    LIMIT 20;
""").fetchdf()

feb_top20_mmsi

,MMSI,n_row,first_ts,last_ts
0,246714000,743865,2023-02-01 00:00:20,2023-02-28 23:59:13
1,255806054,440784,2023-02-03 02:55:27,2023-02-19 19:20:41
2,563077060,328479,2023-02-01 00:00:08,2023-02-28 23:58:58
3,370454000,318885,2023-02-01 00:00:16,2023-02-28 23:59:57
4,563032960,317551,2023-02-01 00:00:22,2023-02-28 23:59:43
5,563034080,307264,2023-02-01 08:52:23,2023-02-28 23:59:56
6,413263050,296939,2023-02-01 00:00:03,2023-02-28 23:59:53
7,563036650,294884,2023-02-01 00:03:46,2023-02-28 22:48:06
8,563066540,292559,2023-02-01 00:00:00,2023-02-28 23:59:56
9,564461000,288131,2023-02-01 00:00:04,2023-02-28 23:59:51


In [7]:
aug_top20_mmsi = con.execute("""
    SELECT MMSI,
           COUNT(*) AS n_row,
           MIN("TIMESTAMP") AS first_ts,
           MAX("TIMESTAMP") AS last_ts
    FROM aug_ais
    WHERE MMSI IS NOT NULL AND MMSI <> 0
    GROUP BY MMSI
    ORDER BY n_row DESC
    LIMIT 20;
""").fetchdf()

aug_top20_mmsi

,MMSI,n_row,first_ts,last_ts
0,255806054,654225,2023-08-03 14:14:39,2023-08-20 16:13:29
1,235108526,477523,2023-08-01 14:53:29,2023-08-26 19:51:13
2,563036630,418953,2023-08-01 00:00:01,2023-08-31 23:59:53
3,563142000,357187,2023-08-01 00:00:01,2023-08-31 23:59:57
4,566623000,342977,2023-08-01 00:00:28,2023-08-31 23:59:56
5,566687000,332129,2023-08-01 00:00:14,2023-08-31 23:59:57
6,563480000,319251,2023-08-01 00:00:05,2023-08-31 23:59:58
7,565081000,314477,2023-08-01 00:00:04,2023-08-31 23:59:53
8,664104000,313930,2023-08-06 03:53:33,2023-08-31 23:59:37
9,563036650,308654,2023-08-01 02:04:58,2023-08-31 23:59:46


In [8]:
export_mmsi_to_parquet(con, "feb_ais", 246714000)
export_mmsi_to_parquet(con, "aug_ais", 255806054)

Exported MMSI 246714000 → output\mmsi_246714000.parquet
Exported MMSI 255806054 → output\mmsi_255806054.parquet


In [9]:
list_file_size("output")

output\mmsi_246714000.parquet: 1.43 MB
output\mmsi_255806054.parquet: 0.75 MB


In [10]:
con.execute("""
    COPY (
        SELECT *, (MMSI % 100) AS mmsi_bucket
        FROM feb_ais
        WHERE MMSI IS NOT NULL AND MMSI <> 0
    )
    TO 'output/feb_mmsi_bucket'
    (FORMAT PARQUET, PARTITION_BY (mmsi_bucket), COMPRESSION ZSTD);
""")

In [11]:
con.execute("""
    COPY (
        SELECT *, (MMSI % 100) AS mmsi_bucket
        FROM aug_ais
        WHERE MMSI IS NOT NULL AND MMSI <> 0
    )
    TO 'output/aug_mmsi_bucket'
    (FORMAT PARQUET, PARTITION_BY (mmsi_bucket), COMPRESSION ZSTD);
""")

In [12]:
list_file_size("output/feb_mmsi_bucket")

output/feb_mmsi_bucket\mmsi_bucket=0\data_0.parquet: 565.01 MB
output/feb_mmsi_bucket\mmsi_bucket=1\data_0.parquet: 6.23 MB
output/feb_mmsi_bucket\mmsi_bucket=10\data_0.parquet: 15.98 MB
output/feb_mmsi_bucket\mmsi_bucket=11\data_0.parquet: 4.93 MB
output/feb_mmsi_bucket\mmsi_bucket=12\data_0.parquet: 2.12 MB
output/feb_mmsi_bucket\mmsi_bucket=13\data_0.parquet: 2.77 MB
output/feb_mmsi_bucket\mmsi_bucket=14\data_0.parquet: 3.01 MB
output/feb_mmsi_bucket\mmsi_bucket=15\data_0.parquet: 4.33 MB
output/feb_mmsi_bucket\mmsi_bucket=16\data_0.parquet: 4.08 MB
output/feb_mmsi_bucket\mmsi_bucket=17\data_0.parquet: 3.22 MB
output/feb_mmsi_bucket\mmsi_bucket=18\data_0.parquet: 4.25 MB
output/feb_mmsi_bucket\mmsi_bucket=19\data_0.parquet: 3.26 MB
output/feb_mmsi_bucket\mmsi_bucket=2\data_0.parquet: 3.30 MB
output/feb_mmsi_bucket\mmsi_bucket=20\data_0.parquet: 29.65 MB
output/feb_mmsi_bucket\mmsi_bucket=21\data_0.parquet: 3.00 MB
output/feb_mmsi_bucket\mmsi_bucket=22\data_0.parquet: 3.85 MB
output/

In [13]:
list_file_size("output/aug_mmsi_bucket")

output/aug_mmsi_bucket\mmsi_bucket=0\data_0.parquet: 756.77 MB
output/aug_mmsi_bucket\mmsi_bucket=1\data_0.parquet: 8.98 MB
output/aug_mmsi_bucket\mmsi_bucket=10\data_0.parquet: 33.47 MB
output/aug_mmsi_bucket\mmsi_bucket=11\data_0.parquet: 5.79 MB
output/aug_mmsi_bucket\mmsi_bucket=12\data_0.parquet: 4.70 MB
output/aug_mmsi_bucket\mmsi_bucket=13\data_0.parquet: 5.86 MB
output/aug_mmsi_bucket\mmsi_bucket=14\data_0.parquet: 4.63 MB
output/aug_mmsi_bucket\mmsi_bucket=15\data_0.parquet: 4.37 MB
output/aug_mmsi_bucket\mmsi_bucket=16\data_0.parquet: 4.64 MB
output/aug_mmsi_bucket\mmsi_bucket=17\data_0.parquet: 5.51 MB
output/aug_mmsi_bucket\mmsi_bucket=18\data_0.parquet: 5.92 MB
output/aug_mmsi_bucket\mmsi_bucket=19\data_0.parquet: 4.48 MB
output/aug_mmsi_bucket\mmsi_bucket=2\data_0.parquet: 3.18 MB
output/aug_mmsi_bucket\mmsi_bucket=20\data_0.parquet: 46.12 MB
output/aug_mmsi_bucket\mmsi_bucket=21\data_0.parquet: 4.51 MB
output/aug_mmsi_bucket\mmsi_bucket=22\data_0.parquet: 4.93 MB
output/